In [63]:
import tdmclient.notebook
from pylsl import StreamInlet, resolve_stream
import numpy as np
import time

In [65]:
def connectStream():
        # first resolve an EEG stream on the lab network
        print("looking for an EEG stream...")
        streams = resolve_stream('type', 'EEG')


        # create a new inlet to read from the stream
        inlet = StreamInlet(streams[0])

        print('connection found')
        return inlet

In [64]:
await tdmclient.notebook.start()

In [31]:


try : inlet = connectStream()
except : print('No stream found')

looking for an EEG stream...
connection found


Code pour suivre la ligne 

In [62]:
%%run_aseba
var speed_left
var speed_right
#var target_speed
var left
var right
# mean and vari(ance) calculation vars
var mini
var maxi
var mean
var vari
# actual value and n(ormalized)dev(iation)
var p1
var ndev
# pi controller vars
var ireg
var preg
var pcoeff
var icoeff

var SIGMA
#onevent startup
  mini=1023
  maxi=0
  mean=512
  vari=512
  ireg=0

  SIGMA = 100

onevent setSpeed s
   speed_left = s
   speed_right = s

onevent prox
   # speed_left=motor.left.target #or another value
   # speed_right=motor.right.target #or another value
   p1=prox.ground.delta[1]
   callsub statistics 
   call math.muldiv(ndev,SIGMA,p1-mean,vari)
   if abs(ndev)<SIGMA then
      preg=60*ndev/100    
      ireg=ireg+10*preg/30
      motor.left.target=speed_left+(preg+ireg)            
      motor.right.target=speed_right-(preg+ireg)         
   else    
      ireg=0                
      motor.left.target=1*ndev/2
      motor.right.target=-1*ndev/2
   end

sub statistics
   call math.max(maxi,maxi,p1)
   call math.min(mini,mini,p1)        
   if maxi-mini>400 then
      call math.muldiv(vari,45,maxi-mini,100)
      mean=(mini+maxi)/2
   end


KeyError: 'error_msg'

In [52]:
%%run_python
@onevent
def sendSpeed(speed):
    global motor_left_target, motor_right_target
    motor_left_target=speed
    motor_right_target=speed
    # leds_top=GREEN

In [56]:
send_event('sendSpeed',0)

In [61]:
while True:
    # get a new sample (you can also omit the timestamp part if you're not
    # interested in it)
    sample, timestamp = inlet.pull_sample()
    if timestamp:
        alpha = sample[0]
        # alpha = valeur actuelle - baseline
        # donc si alpha < 0 : plus concentré = on augmente la vitesse
        # sinon on réduit la vitesse
        if alpha <0 : 
            speed = speed + min(-alpha*5, 400)
        else :
            speed =  speed - max(alpha*5, 0)
        
        print(f'Alpha : {alpha}')
        print(f'Speed : {speed}\n')
        send_event("sendSpeed",int(speed))
        

Alpha : 17.31943702697754
Speed : 21.80132120847702

Alpha : 21.51955795288086
Speed : 21.80132120847702

Alpha : 14.948988914489746
Speed : 21.80132120847702

Alpha : 15.736991882324219
Speed : 21.80132120847702

Alpha : 19.581846237182617
Speed : 21.80132120847702

Alpha : 22.123905181884766
Speed : 21.80132120847702

Alpha : 21.58184051513672
Speed : 21.80132120847702

Alpha : 21.920063018798828
Speed : 21.80132120847702

Alpha : 33.5244140625
Speed : 21.80132120847702

Alpha : 38.35225296020508
Speed : 21.80132120847702

Alpha : 39.9571533203125
Speed : 21.80132120847702

Alpha : 34.14481735229492
Speed : 21.80132120847702

Alpha : 35.35710906982422
Speed : 21.80132120847702

Alpha : 37.052337646484375
Speed : 21.80132120847702

Alpha : 38.7866325378418
Speed : 21.80132120847702

Alpha : 39.33584976196289
Speed : 21.80132120847702

Alpha : 29.923494338989258
Speed : 21.80132120847702

Alpha : 42.523170471191406
Speed : 21.80132120847702

Alpha : 44.99599838256836
Speed : 21.8013212

LostError: the stream has been lost.